## 도구 준비

In [1]:
import re
import glob
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

## 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.']


## 데이터 전처리

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내세요!



In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

The Cat in the Hat
By Dr. Seuss
The sun did not shine.
It was too wet to play.
So we sat in the house
All that cold cold wet day.
I sat there with Sally.
We sat there we two.
And I said How I wish
We had something to do!


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
#    if len(preprocess_sentence(sentence)) <= 15: continue
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen = 15, padding='post')  
#    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') 
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  903 ...    0    0    0]
 [   2  122 2584 ...    0    0    0]
 [   2    6  304 ...    0    0    0]
 ...
 [ 673   27    6 ...    6  189    3]
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4e197483d0>


In [7]:
#데이터 길이 확인
len(tensor[1])

15

In [8]:
print(tensor[:3, :10])

[[   2    6  903   14    6 1350    3    0    0    0]
 [   2  122 2584   20    1    3    0    0    0    0]
 [   2    6  304  166   70  559   20    3    0    0]]


In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [10]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0]
[   6  903   14    6 1350    3    0    0    0    0    0    0    0    0]


In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [12]:
#데이터 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [13]:
#모델 설계
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.01581586e-05,  3.00796499e-04, -2.98221625e-04, ...,
          1.12974871e-04,  3.33018863e-04,  2.06015466e-05],
        [-1.33034060e-04,  7.64945522e-04, -7.71292776e-04, ...,
          4.85848068e-06,  5.25960408e-04,  9.00514424e-05],
        [-1.97639340e-04,  7.59615155e-04, -1.13645697e-03, ...,
          2.51424663e-05,  6.45792112e-04, -1.66653859e-04],
        ...,
        [ 1.74648550e-04,  2.01610313e-03, -2.20868527e-03, ...,
          2.62596970e-03, -3.76730575e-04,  2.08904912e-05],
        [ 3.99907614e-04,  2.45456211e-03, -2.40178546e-03, ...,
          3.33099905e-03, -1.00937916e-03,  7.47312224e-05],
        [ 6.28689362e-04,  2.82754074e-03, -2.58201710e-03, ...,
          3.96129908e-03, -1.65014458e-03,  1.55535075e-04]],

       [[-3.01581586e-05,  3.00796499e-04, -2.98221625e-04, ...,
          1.12974871e-04,  3.33018863e-04,  2.06015466e-05],
        [-1.33034060e-04,  7.64945522e-04, -7

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 104s 152ms/step - loss: 3.5606
Epoch 2/10
686/686 [==============================] - 105s 152ms/step - loss: 3.0675
Epoch 3/10
686/686 [==============================] - 104s 152ms/step - loss: 2.8628
Epoch 4/10
686/686 [==============================] - 104s 151ms/step - loss: 2.7010
Epoch 5/10
686/686 [==============================] - 105s 153ms/step - loss: 2.5567
Epoch 6/10
686/686 [==============================] - 105s 152ms/step - loss: 2.4226
Epoch 7/10
686/686 [==============================] - 105s 153ms/step - loss: 2.2950
Epoch 8/10
686/686 [==============================] - 104s 152ms/step - loss: 2.1737
Epoch 9/10
686/686 [==============================] - 104s 152ms/step - loss: 2.0570
Epoch 10/10
686/686 [==============================] - 105s 153ms/step - loss: 1.9442


256, 1024, loss: 2.2301
512, 1024, loss: 1.9442

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you lie <end> '

# Conclusion
### 데이터 전처리
이번 프로젝트에서 가장 난이도가 있던 부분은 데이터 전처리이다. 가장 애매한 부분은 어떤 문자는 남겨놓고 어떤 문자는 버릴지 결정하는 일이다. 예를 들어 특수문자 '('같은 경우 사용하는 경우는 드물지만 전혀 안 쓴다고 볼 수는 없으며 이번 프로젝트에서 알파벳를 비롯한 몇몇 특수문자를 제외하곤 제거했는데 résumé와 같이 외국어를 그대로 표기하는 경우 필요한 데이터가 날아갈 가능성이 높다.
이는 한글에선 더 큰 문제인데 동음이의어나 잘 사용되지 않는 단어의 경우 주석(註釋)과 같은 형태로 한글 + 외국어의 형태로 표기하는 경우가 종종 존재하며 아예 영어와 병기하는 경우도 있기 때문이다. 이러한 이유로 입력 데이터의 종류와 성격에 따라서 데이터 전처리에 있어서 신중을 기해야 한다.
### 모델 설계
모델의 경우 자연어처리에 있어서 뛰어난 성능을 가졌다고 알려진 LSTM이 사용되었다.
파라미터 설정의 경우 hidden size를 고정값으로 두고 embedding size는 각각 256, 512를 사용했으며 epoch는 10을 주었다.
### 총평
embedding size 256, hidden size 1024, epoch 10을 주었을때 loss는 2.23이었으며 i love를 입력 데이터로 주었을때 '<start> i love cease sunset emerald hairy lots lots reminisce reminisce reminisce hypnotized hypnotized fantastic fantastic fantastic forward kindly kindly '가 출력되었다.
반면 embedding size 512, hidden size 1024, epoch 10을 주었을때 loss는 1.94였고 출력 데이터는 '<start> i love the way you lie <end> '였다.
생각보다 훨씬 완성도가 높은 결과물이 나왔다.